In [ ]:
# Recall test from JSON file 

import ollama
import json
import os

MODEL = "llama3.1:8b"
MEMORY_FILE = r"update path"

def test_extended_recall():
    # Load manual JSON file
    if not os.path.exists(MEMORY_FILE):
        print("Error: Please create test_profile.json first")
        return

    with open(MEMORY_FILE, 'r') as f:
        memory = json.load(f)
    
    print(f"Loaded Memory for: {memory['name']}")

    # Inject Memory into System Prompt
    system_prompt = f"""
    You are a Career Advisor.
    
    USER PROFILE (Long Term Memory):
    Name: {memory['name']}
    College: {memory['college']}
    Skills: {memory['skills']}
    Target Goal: {memory['goal']}
    
    Instructions:
    Answer questions based on the User Profile.
    """

    # List of questions to test different memory fields
    questions = [
        "Based on my profile, what specific job roles should I apply for?",
        "Which college am I from?",
        "Do I have any experience with Python?"
    ]

    # Initialize chat history with system prompt
    messages = [{'role': 'system', 'content': system_prompt}]

    print("Starting Memory Check...")

    for i, user_query in enumerate(questions, 1):
        print(f"\nTest {i}: {user_query}")
        
        # Add user question to history
        messages.append({'role': 'user', 'content': user_query})
        
        # Get response
        response = ollama.chat(model=MODEL, messages=messages)
        answer = response['message']['content']
        
        print(f"Agent: {answer}")
        
        # Add agent answer to history so it remembers the conversation flow
        messages.append({'role': 'assistant', 'content': answer})

if __name__ == "__main__":
    test_extended_recall()

Loaded Memory for: Hari
Starting Memory Check...

Test 1: Based on my profile, what specific job roles should I apply for?
Agent: Based on your skills and target goal, here are some specific job roles that you may be suited for:

1. **Machine Learning Engineer**: With expertise in Machine Learning, Python, and Signal Processing, you can explore opportunities as a Machine Learning Engineer in industries like tech, finance, or healthcare.
2. **AI/ML Research Scientist**: As an AI enthusiast with skills in Machine Learning and Signal Processing, you may be interested in research-oriented roles where you can work on cutting-edge projects and publications.
3. **Signal Processing Engineer**: With your expertise in Signal Processing, you can explore opportunities in industries like audio engineering, image processing, or telecommunications.
4. **Data Scientist (AI/ML focus)**: As a data scientist with expertise in Machine Learning, Python, and Signal Processing, you can work on projects invol

In [ ]:
import ollama
import json
import os

MODEL = "llama3.1:8b"
# Update this path to your specific folder
SAVE_DIR = r"update path"
OUTPUT_FILE = os.path.join(SAVE_DIR, "generated_profile.json")

def test_creation_and_verification():
    # Ensure the directory exists
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    print("Step 1: Simulating Conversation...")
    
    # 1. Simulate Chat History
    chat_history = [
        "Hi, I'm Sarah.",
        "I am a final year student at IIT Madras.",
        "I know Java and Spring Boot, but I want to get into Cloud Computing.",
        "I have a certification in AWS."
    ]
    
    full_conversation_text = "\n".join(chat_history)
    print(f"Chat Input:\n{full_conversation_text}\n")
    
    # 2. Extract Data to JSON
    system_prompt_extractor = """
    You are a Data Extractor.
    Read the conversation history and extract the user's details into a JSON object.
    Fields required: "name", "college", "skills", "certifications", "interest".
    Output strictly valid JSON only.
    """
    
    print("Step 2: Extracting Profile to JSON...")
    response = ollama.chat(model=MODEL, messages=[
        {'role': 'system', 'content': system_prompt_extractor},
        {'role': 'user', 'content': full_conversation_text}
    ])
    
    raw_output = response['message']['content']
    
    # Parse JSON
    try:
        if "```json" in raw_output:
            json_str = raw_output.split("```json")[1].split("```")[0].strip()
        elif "```" in raw_output:
            json_str = raw_output.split("```")[1].split("```")[0].strip()
        else:
            json_str = raw_output.strip()
            
        data = json.loads(json_str)
        
        # Save to file
        with open(OUTPUT_FILE, 'w') as f:
            json.dump(data, f, indent=4)
            
        print(f"Success. Profile saved to: {OUTPUT_FILE}")
        
    except Exception as e:
        print(f"Failed to parse JSON. Error: {e}")
        return

    # 3. Verify Memory (Read the file back and ask questions)
    print("\nStep 3: Verifying Memory (Asking questions based on the new file)...")
    
    # Load the file we just saved
    with open(OUTPUT_FILE, 'r') as f:
        memory = json.load(f)

    # Create a new agent instance with this memory
    system_prompt_agent = f"""
    You are a Career Assistant.
    USER PROFILE:
    Name: {memory.get('name')}
    College: {memory.get('college')}
    Skills: {memory.get('skills')}
    Certifications: {memory.get('certifications')}
    Interest: {memory.get('interest')}
    
    Answer questions based strictly on this profile.
    """

    verification_questions = [
        "What is the candidate's name and college?",
        "Does she have any cloud certifications?",
        "Based on her interests, what job role would you suggest?"
    ]

    messages = [{'role': 'system', 'content': system_prompt_agent}]

    for q in verification_questions:
        print(f"\nQ: {q}")
        messages.append({'role': 'user', 'content': q})
        
        response = ollama.chat(model=MODEL, messages=messages)
        answer = response['message']['content']
        
        print(f"A: {answer}")
        messages.append({'role': 'assistant', 'content': answer})

if __name__ == "__main__":
    test_creation_and_verification()

Step 1: Simulating Conversation...
Chat Input:
Hi, I'm Sarah.
I am a final year student at IIT Madras.
I know Java and Spring Boot, but I want to get into Cloud Computing.
I have a certification in AWS.

Step 2: Extracting Profile to JSON...
Success. Profile saved to: C:\Users\hari7\Documents\Anokha Hackthon\file_samples\generated_profile.json

Step 3: Verifying Memory (Asking questions based on the new file)...

Q: What is the candidate's name and college?
A: The candidate's name is Sarah, and she attends IIT Madras.

Q: Does she have any cloud certifications?
A: Yes, Sarah has an AWS certification.

Q: Based on her interests, what job role would you suggest?
A: Given that Sarah's interest lies in Cloud Computing and she already has an AWS certification, I would suggest a role such as a Cloud Engineer or a DevOps Engineer with expertise in designing, deploying, and managing cloud-based systems. Her Java and Spring Boot skills could be valuable assets in this role.
